# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#set csv file
weather_csv = "../output_data/cities.csv"

#read csv into dataframe
weather_df = pd.read_csv(weather_csv)

weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,saskylakh,71.9167,114.0833,-22.32,84,0,6.64,RU,1613012657
1,lucapa,-8.4192,20.7447,64.81,96,100,2.68,AO,1613012651
2,palmer,42.1584,-72.3287,16.00,73,90,1.83,US,1613012060
3,vardo,70.3705,31.1107,24.33,86,100,13.35,NO,1613012760
4,puerto ayora,-0.7393,-90.3518,75.00,99,79,1.01,EC,1613012762
...,...,...,...,...,...,...,...,...,...
573,verkhnevilyuysk,63.4453,120.3167,-15.57,88,93,2.66,RU,1613013465
574,camacari,-12.6975,-38.3242,82.40,69,40,11.50,BR,1613013466
575,bayburt,40.2589,40.2278,22.73,93,2,1.90,TR,1613013467
576,paris,48.8534,2.3488,23.00,80,0,10.36,FR,1613012930


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Store humidity and convert to float
humidity = weather_df["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(0,0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
vacay_weather_df = weather_df.loc[(weather_df["Max Temp"]>=65) & (weather_df["Max Temp"]<=80), :]
vacay_weather_df = vacay_weather_df.dropna()
vacay_weather_df.reset_index(inplace=True)
vacay_weather_df = vacay_weather_df.head(10)
vacay_weather_df

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,4,puerto ayora,-0.7393,-90.3518,75.00,99,79,1.01,EC,1613012762
1,7,ushuaia,-54.8000,-68.3000,66.20,45,40,13.80,AR,1613012449
2,13,east london,-33.0153,27.9116,73.92,90,1,9.64,ZA,1613012772
3,22,hilo,19.7297,-155.0900,77.00,57,20,10.36,US,1613012711
4,26,cape town,-33.9258,18.4232,68.00,88,0,13.80,ZA,1613012709
5,30,pastos bons,-6.6017,-44.0767,71.87,92,72,1.74,BR,1613012792
6,33,rikitea,-23.1203,-134.9692,78.48,78,3,13.22,PF,1613012796
7,35,saint-pierre,-21.3393,55.4781,77.00,69,0,4.61,RE,1613012435
8,36,kapoeta,4.7717,33.5903,78.91,36,100,7.16,SS,1613012710
9,39,acapulco,16.8634,-99.8901,78.80,73,20,6.91,MX,1613012803


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = vacay_weather_df.loc[:, ["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
0,puerto ayora,EC,-0.7393,-90.3518,
1,ushuaia,AR,-54.8000,-68.3000,
2,east london,ZA,-33.0153,27.9116,
3,hilo,US,19.7297,-155.0900,
4,cape town,ZA,-33.9258,18.4232,
5,pastos bons,BR,-6.6017,-44.0767,
6,rikitea,PF,-23.1203,-134.9692,
7,saint-pierre,RE,-21.3393,55.4781,
8,kapoeta,SS,4.7717,33.5903,
9,acapulco,MX,16.8634,-99.8901,


In [7]:
# find the closest hotel for each vacation spot
params = {
    "radius": 5000,
    "type": "lodging",
    "keyword": "hotel",
    "key": g_key,
}


# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))
    

    # Skip data that may be missing
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,puerto ayora,EC,-0.7393,-90.3518,La Isla Hotel
1,ushuaia,AR,-54.8000,-68.3000,Patagonia Jarke
2,east london,ZA,-33.0153,27.9116,Blue Lagoon Hotel
3,hilo,US,19.7297,-155.0900,Hilo Hawaiian Hotel
4,cape town,ZA,-33.9258,18.4232,The Silo Hotel
5,pastos bons,BR,-6.6017,-44.0767,Hotel Favorito
6,rikitea,PF,-23.1203,-134.9692,Pension Bianca & Benoit
7,saint-pierre,RE,-21.3393,55.4781,"Villa Delisle, Hotel & Spa"
8,kapoeta,SS,4.7717,33.5903,Kuleu Hotel
9,acapulco,MX,16.8634,-99.8901,Torres Gemelas


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))